In [ ]:
!pip install datasets peft transformers==4.30
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 31.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB

In [ ]:

from google.colab import files


uploaded = files.upload()

Saving finalDoctors.csv to finalDoctors.csv


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, DataCollatorWithPadding, Trainer, TrainingArguments, BertForSequenceClassification, pipeline
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, get_peft_model
import torch
import pandas as pd
import numpy as np
import os

In [ ]:
data_files = {"train": "train.jsonl", "test": "test.jsonl"}
dataset = load_dataset("gretelai/symptom_to_diagnosis", data_files=data_files)
dataset = dataset.rename_column("output_text", "label")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/212 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_text'],
        num_rows: 853
    })
    test: Dataset({
        features: ['label', 'input_text'],
        num_rows: 212
    })
})


In [ ]:
for entry in dataset['train'].select(range(5)):
    print('INPUT: {} \nOUTPUT: {}\n'.format(entry['input_text'], entry['label']))

INPUT: I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak. 
OUTPUT: cervical spondylosis

INPUT: I have a rash on my face that is getting worse. It is red, inflamed, and has blisters that are bleeding clear pus. It is really painful. 
OUTPUT: impetigo

INPUT: I have been urinating blood. I sometimes feel sick to my stomach when I urinate. I often feel like I have a fever. 
OUTPUT: urinary tract infection

INPUT: I have been having trouble with my muscles and joints. My neck is really tight and my muscles feel weak. I have swollen joints and it is hard to move around without becoming stiff. It is also really uncomfortable to walk. 
OUTPUT: arthritis

INPUT: I have been feeling really sick. My body hurts a lot and I have no appetite. I have also developed rashes on my arms and face. The back of my eyes hurt a lot. 
OUTPUT: dengue



In [ ]:
train_counts = pd.DataFrame({'Diagnosis': dataset['train']['label']})
train_counts = train_counts.groupby('Diagnosis').size().reset_index(name='train_set')

test_counts = pd.DataFrame({'Diagnosis': dataset['test']['label']})
test_counts = test_counts.groupby('Diagnosis').size().reset_index(name='test_set')

display(train_counts.merge(test_counts, on='Diagnosis'))

,Diagnosis,train_set,test_set
0,allergy,40,10
1,arthritis,40,10
2,bronchial asthma,40,10
3,cervical spondylosis,40,10
4,chicken pox,40,10
5,common cold,39,10
6,dengue,40,10
7,diabetes,40,10
8,drug reaction,40,8
9,fungal infection,39,9


In [ ]:
sorted_labels = sorted(set(dataset['train']['label']))
label2id = dict(zip(sorted_labels, range(0, len(sorted_labels))))
id2label = dict(zip(range(0, len(sorted_labels)), sorted_labels))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
foundation_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                                      num_labels=len(label2id),
                                                                      label2id=label2id,
                                                                      id2label=id2label)

classifier = pipeline("text-classification", model=foundation_model, tokenizer=tokenizer)
predicted_labels = classifier(dataset['test']['input_text'])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_array = np.asarray(dataset['test']['label'])
pred_array = np.asarray([item['label'] for item in predicted_labels])
foundation_accuracy = round(sum(test_array == pred_array)*100/len(test_array), 2)
print(f"Foundation Model Accuracy: {foundation_accuracy}%")

Foundation Model Accuracy: 4.72%


In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=64, lora_alpha=1, lora_dropout=0.1
)

peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.bert)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=64, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=64, out_features=768, bias=False)
              )
              (lora_embedding_A): P

In [ ]:
peft_model.print_trainable_parameters()


trainable params: 2,376,214 || all params: 111,875,372 || trainable%: 2.1239831050572953


In [ ]:
def preprocess_function(examples):

    tokens = tokenizer(examples["input_text"], padding="max_length", truncation=True)
    tokens['label'] = [label2id[l] for l in examples["label"]]
    return tokens

splits = ['train', 'test']

tokenized_ds = {}

for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

print(tokenized_ds)

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['label', 'input_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 853
}), 'test': Dataset({
    features: ['label', 'input_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 212
})}


In [ ]:
print("A tokenized training input example:")
print(tokenized_ds["train"][0]["input_ids"])
print("\n")
print("A tokenized training label example:")
print(tokenized_ds["train"][0]["label"])

A tokenized training input example:
[101, 1045, 1005, 2310, 2042, 2383, 1037, 2843, 1997, 3255, 1999, 2026, 3300, 1998, 2067, 1012, 1045, 1005, 2310, 2036, 2042, 2383, 4390, 2007, 2026, 5703, 1998, 12016, 1012, 1045, 1005, 2310, 2042, 21454, 1037, 2843, 1998, 2026, 10726, 2514, 5410, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()*100}


trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="bert-lora",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=15,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print("Starting to train...")
trainer.train()

Starting to train...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.388680,27.830189
2,No log,0.938292,72.641509
3,2.109800,0.751096,75.000000
4,2.109800,0.539223,83.962264
5,0.503500,0.316643,89.622642
6,0.503500,0.369825,89.622642
7,0.503500,0.299454,92.452830
8,0.170700,0.248775,93.867925
9,0.170700,0.186091,94.339623
10,0.056300,0.235381,93.867925


TrainOutput(global_step=3210, training_loss=0.4487255172937459, metrics={'train_runtime': 1121.1626, 'train_samples_per_second': 11.412, 'train_steps_per_second': 2.863, 'total_flos': 3460510521077760.0, 'train_loss': 0.4487255172937459, 'epoch': 15.0})

In [ ]:
peft_bert_model_path = "fine-tuned-peft-model-weights/"
peft_model.save_pretrained(peft_bert_model_path)

# check the size of the saved model
for file_name in os.listdir(peft_bert_model_path):
    file_size = os.path.getsize(peft_bert_model_path + file_name)
    print(f"File Name: {file_name}; File Size: {file_size / 1024:.2f}KB")

File Name: adapter_config.json; File Size: 0.62KB
File Name: README.md; File Size: 4.97KB
File Name: adapter_model.safetensors; File Size: 9288.92KB


In [ ]:
config = PeftConfig.from_pretrained('fine-tuned-peft-model-weights/')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=22)
model = PeftModel.from_pretrained(model, 'fine-tuned-peft-model-weights/')

trainer = Trainer(
    model=model,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_predictions = trainer.predict(tokenized_ds['test'])


In [ ]:
input_text = input()
input_text_tokenized = tokenizer.encode(input_text,
                                        truncation=True,
                                        padding=True,
                                        return_tensors="pt").to("cuda:0")

with torch.no_grad():
    outputs = model(input_text_tokenized)

Headache and nausea


In [ ]:

predicted_label_id = torch.argmax(outputs.logits).item()
pred = train_counts.iloc[predicted_label_id]['Diagnosis']
pred

'hypertension'

In [ ]:
import torch
# predictions = torch.sigmoid(output.logits)
# predicted_labels = [model.config.id2label[_id] for _id in (predictions > 0.3).nonzero()[:, 1].tolist()]
# predicted_labels


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("/content/finalDoctors.csv")

In [ ]:
df

,Unnamed: 0,Name,Disease Specialization,Rating,starts,ends
0,0,John Smith,Stroke,4.3,8,20
1,1,Sarah Lee,Epilepsy,4.7,8,20
2,2,Michael Chen,Pediatrics,4.5,7,18
3,3,Emily Davis,Dermatitis,4.1,9,17
4,4,Jason Kim,Orthopedic injuries,4.4,10,23
...,...,...,...,...,...,...
171,171,Olivia Nguyen,Stroke,4.4,8,18
172,172,Ethan Johnson,Gynecologic cancers,4.2,10,19
173,173,David Kim,Chronic rhinitis,4.6,7,14
174,174,Julia Brown,Kidney stones,4.3,8,22


In [ ]:
from sentence_transformers import SentenceTransformer
sentences = df['Disease Specialization']

model = SentenceTransformer('aleynahukmet/bge-medical-small-en-v1.5')
embeddings = model.encode(sentences)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
embs = []
for i in range(len(embeddings)):
  embs.append(embeddings[i])

df = df.assign(embeddings = embs)


In [ ]:
df

,Unnamed: 0,Name,Disease Specialization,Rating,starts,ends,embeddings
0,0,John Smith,Stroke,4.3,8,20,"[0.008636835, 0.028790627, -0.00033120092, 0.0..."
1,1,Sarah Lee,Epilepsy,4.7,8,20,"[-0.028133608, 0.006354786, 0.018376548, 0.058..."
2,2,Michael Chen,Pediatrics,4.5,7,18,"[-0.012473587, -0.092413016, 0.021856526, 0.01..."
3,3,Emily Davis,Dermatitis,4.1,9,17,"[0.05954571, -0.066363916, 0.08311083, -0.0117..."
4,4,Jason Kim,Orthopedic injuries,4.4,10,23,"[-0.06022175, -0.03773368, 0.010702274, -0.084..."
...,...,...,...,...,...,...,...
171,171,Olivia Nguyen,Stroke,4.4,8,18,"[0.008636835, 0.028790627, -0.00033120092, 0.0..."
172,172,Ethan Johnson,Gynecologic cancers,4.2,10,19,"[-0.028035156, 0.027226869, -0.052928794, -0.0..."
173,173,David Kim,Chronic rhinitis,4.6,7,14,"[0.07966374, -0.03269891, 0.020735675, 0.05608..."
174,174,Julia Brown,Kidney stones,4.3,8,22,"[0.04649128, -0.049787167, 0.048849914, 0.0648..."


In [ ]:
start = int(input())
end = int(input())
X = df[((df['starts'] <= start) & (df['ends'] >= end))]
X

10
16


,Unnamed: 0,Name,Disease Specialization,Rating,starts,ends,embeddings
0,0,John Smith,Stroke,4.3,8,20,"[0.008636835, 0.028790627, -0.00033120092, 0.0..."
1,1,Sarah Lee,Epilepsy,4.7,8,20,"[-0.028133608, 0.006354786, 0.018376548, 0.058..."
2,2,Michael Chen,Pediatrics,4.5,7,18,"[-0.012473587, -0.092413016, 0.021856526, 0.01..."
3,3,Emily Davis,Dermatitis,4.1,9,17,"[0.05954571, -0.066363916, 0.08311083, -0.0117..."
4,4,Jason Kim,Orthopedic injuries,4.4,10,23,"[-0.06022175, -0.03773368, 0.010702274, -0.084..."
...,...,...,...,...,...,...,...
170,170,Austin Lee,Chronic pancreatitis,4.1,8,16,"[0.0673338, -0.043415044, 0.011190913, 0.06241..."
171,171,Olivia Nguyen,Stroke,4.4,8,18,"[0.008636835, 0.028790627, -0.00033120092, 0.0..."
172,172,Ethan Johnson,Gynecologic cancers,4.2,10,19,"[-0.028035156, 0.027226869, -0.052928794, -0.0..."
174,174,Julia Brown,Kidney stones,4.3,8,22,"[0.04649128, -0.049787167, 0.048849914, 0.0648..."


In [ ]:
# top_preds = predicted_labels[:5]


# icd9codes = map(lambda x : str(search(x)).split(":")[1], top_preds)
top = pred
top_emb = model.encode(top)
top_emb

array([ 5.72371781e-02, -3.97716044e-03, -1.89489946e-02,  1.14984781e-01,
       -4.97522689e-02,  2.72291843e-02, -2.28349231e-02, -1.41598862e-02,
        5.89373149e-02, -2.49339398e-02,  6.21391870e-02,  5.37862852e-02,
        4.21000470e-04, -2.04629432e-02, -1.80308353e-02,  6.94260513e-03,
       -9.49152708e-02, -1.64980162e-02, -9.46151540e-02,  2.27138004e-03,
        4.92894724e-02, -1.02872793e-02,  6.64650947e-02, -4.27537262e-02,
        1.71381216e-02, -4.36384715e-02, -5.25258109e-02, -8.03168938e-02,
        4.17008717e-03,  3.52447443e-02,  2.65577715e-02,  9.92234517e-03,
        8.75085369e-02, -4.53675129e-02,  7.01310858e-02,  1.70193892e-02,
       -1.23880738e-02, -2.95728035e-02,  2.58980785e-02,  7.29366764e-02,
        5.87877706e-02,  6.23272359e-02,  1.83428023e-02, -3.97123061e-02,
        4.11824211e-02,  4.77264374e-02,  7.87206069e-02,  7.92841688e-02,
        1.96488909e-02,  7.03132898e-02, -8.22028667e-02,  1.00327404e-02,
       -1.88543147e-03,  

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
cosine_similarity(top_emb.reshape(1, -1), top_emb.reshape(1, -1))

array([[1.]], dtype=float32)

In [ ]:
cosinesSim = []

for i in range(len(df)):
  if(df.iloc[i]['starts'] >= end or df.iloc[i]['ends'] <= start):
    cosinesSim.append([[-100]])
    continue

  cosinesSim.append(cosine_similarity(df.iloc[i]['embeddings'].reshape(1,-1), top_emb.reshape(1,-1)))


In [ ]:
cs = []
for i in range(len(cosinesSim)):
  cs.append(cosinesSim[i][0][0])
cs = np.array(cs)

In [ ]:
ind = np.argpartition(cs, -4)[-4:]

In [ ]:
ind

array([ 70,  16,  31, 161])

In [ ]:
final = (df.iloc[ind])
final

,Unnamed: 0,Name,Disease Specialization,Rating,starts,ends,embeddings
70,70,Elijah Davis,Addison's disease,4.2,9,20,"[0.01701032, -0.013707826, -0.02753609, 0.1374..."
16,16,Catherine Chen,High blood pressure,4.5,7,17,"[0.044841684, 0.0010257702, 0.03715975, 0.0097..."
31,31,Nathan Lee,Hypertension,4.5,9,17,"[0.057237152, -0.0039771674, -0.018948978, 0.1..."
161,161,David Smith,Hypertension,4.7,9,18,"[0.057237152, -0.0039771674, -0.018948978, 0.1..."


In [ ]:
final.sort_values("Rating", ascending = False)[['Name', 'Disease Specialization', 'Rating']]

,Name,Disease Specialization,Rating
161,David Smith,Hypertension,4.7
16,Catherine Chen,High blood pressure,4.5
31,Nathan Lee,Hypertension,4.5
70,Elijah Davis,Addison's disease,4.2
